In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Make No'] = input_['Make No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        data = {'cssClass': 'form-control',
                'dropdownType': '2',
                'selectedValue': '',
                'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                                      'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                                      'ModelID': 0,
                                      'SkinID': '1',
                                      'Year': f'''{input_.loc[a, 'Year']}'''}}

        try:
            b = 0
            while True:
                b += 1

                try:
                    resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                                         data=json.dumps(data),
                                         headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                                         proxies=get_proxy(),
                                         timeout=(10, 10))
                    
                    if resp.status_code == 200:
                        break
                except:
                    continue

            # = = = = = = = = = = = = = = =
            
            json_ = resp.json()
            
            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(json_['d'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_model_code = html.xpath('//select[@id="modelDropdown"]/option/@value')[1:]

            # = = = = = = = = = = = = = = =

            list_model = html.xpath('//select[@id="modelDropdown"]/option/text()')[1:]

            # = = = = = = = = = = = = = = =

            if not list_model:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Year': input_.loc[a, 'Year'],
                                    'Make No': input_.loc[a, 'Make No'],
                                    'Make': input_.loc[a, 'Make'],
                                    'Make Code': input_.loc[a, 'Make Code'],
                                    'Model No': [i+1 for i in range(len(list_model))],
                                    'Model': list_model,
                                    'Model Code': list_model_code})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Year': input_.loc[a, 'Year'],
                                     'Make No': input_.loc[a, 'Make No'],
                                     'Make': input_.loc[a, 'Make'],
                                     'Make Code': input_.loc[a, 'Make Code']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Year']} > {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Year', 'Make No', 'Model No'],
                                                ascending=[False, True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.model_1-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Year', 'Make No'],
                                            ascending=[False, True],
                                            ignore_index=True)
    output_error.to_excel('./model_1_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：1] - 1970 > 8.BMW
[剩余数量：1000] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1970 > 10.Cadillac
[剩余数量：999] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1970 > 12.Chevrolet
[剩余数量：998] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1971 > 31.Opel
[剩余数量：997] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1971 > 25.Mazda
[剩余数量：996] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1970 > 13.Chrysler
[剩余数量：995] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1971 > 45.Volkswagen
[剩余数量：994] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1970 > 16.Fiat
[剩余数量：993] - [当前时间：09:34:50]

[状态：ok，尝试次数：1] - 1970 > 14.Citroen
[剩余数量：992] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 1970 > 20.International
[剩余数量：991] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 1970 > 3.AM General
[剩余数量：990] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 1970 > 5.Aston Martin
[剩余数量：989] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 1970 > 23.Land Rover
[剩余数量：988] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 1970 > 19.Honda
[剩余数量：987] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 1970 > 22.Jeep
[剩余数量：986] - [当前时间：09:34:51]

[状态：ok，尝试次数：1] - 197